### Ingestion del Archivo "genre.csv"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType)   

In [0]:
genre_schema = StructType([
    StructField("genreId", IntegerType(), False),
    StructField("genreName", StringType(), True)
])

In [0]:
genre_df = spark.read \
    .option("header", True) \
    .schema(genre_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/genre.csv")

#### Paso 2 - Cambiar el nombre de las columnas según lo "requerido"

In [0]:
genre_renamed_df = genre_df \
    .withColumnsRenamed(
        {"genreId": "genre_id", "genreName": "genre_name"})

#### Paso 3 - Agregar la columna "ingestion_date" y "environment" al DataFrame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
genre_final_df = add_ingestion_date(genre_renamed_df) \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))

#### Paso 5 - Escribir datos en el datalake en formato "Parquet"

In [0]:
genre_final_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("movie_silver.genres")

In [0]:
dbutils.notebook.exit("Success")